# Import libraries

In [22]:
import pandas as pd
import numpy as np

# Enable interactive Matplotlib plots in the notebook
%matplotlib qt5


import matplotlib.pyplot as plt
from matplotlib import cm
import os
import astropy.convolution as krn
import scipy.stats as stats
import sys

import matplotlib.pyplot as plt
from matplotlib.widgets import Button



# 2D plot of fixations and raw samples

## 1. plot all raw x,y
## 2. plot all fixations x,y 


In [23]:
def plot2d(df, subj_nr, path, bboxes=False, box_w=False, box_h=False, stimuli=False):
    
    import matplotlib.patches as patches
    import ast
    import matplotlib.image as mpimg

    conditions = ['bags', 'hats', 'sunglasses', 'shoes', 'dresses']


    for cond in conditions:

        plt.figure()
        plt.style.use('ggplot')
        plt.title(f'{cond}, #{subj_nr}')

        # select condition
        a = df[df.stimuli==cond] 
        
        if a.empty:
            # Raise an error or handle the case when the DataFrame is empty
            raise ValueError(f'There is no data for condition: {cond} in file: {subj_nr}')

        raw_h = plt.scatter(a.user_pred_px_x, a.user_pred_px_y, c='orange', alpha=0.5, edgecolors='black')

        # remove no fixations/saccades (zeros)
        
        fix_h = plt.scatter(a.FixXPos[a.FixXPos>0], a.FixYPos[a.FixYPos>0], c='blue', alpha=0.5, edgecolors='black') 

        # plot center fixation dot
        plt.scatter(a.resX.iloc[0]/2, a.resY.iloc[0]/2, c='red')      
              
        
        if bboxes:
            # plot image boxes
            if cond == 'dresses':
                height = 375
            else:
                width = 250
                height = 250
            
            # get coordinates for the first entry
            left_top_coords = a.imageLocations.iloc[0]
            
            # get paths to image names for that frame
            imageNames = a.imageNames.iloc[0]
            
            # Convert string to actual list
            left_top_coords = ast.literal_eval(left_top_coords)
#             print(f'coordinates: {left_top_coords}')
#             print(f'imageNames: {imageNames}')
            imageNames_paths = ast.literal_eval(imageNames)           
                        
            for stim_idx, coord in enumerate(left_top_coords):                
                left = float(coord[0])
                top = float(coord[1])            
               
                rect = patches.Rectangle((left,top),width,height, 
                                    fill = False,
                                    color = "purple",
                                    linewidth = 2)
                
                plt.gca().add_patch(rect)
                
                if stimuli:
#                     print('Plotting stimuli images') # draw stimuli images
                    
                    # Remove the last directory from the path
                    new_path = os.path.dirname(path)
                    
                    # Fix image name if ends on 'png'
                    if imageNames_paths[stim_idx][-3:] == 'png':
                        imageNames_paths[stim_idx] = imageNames_paths[stim_idx].strip('png') + 'jpg'
                    
                    # Join the main path with path to stimuli
                    new_path = new_path + imageNames_paths[stim_idx][1:]
                    
                    # Load the images
                    image = mpimg.imread(new_path)

                    # Define the extent (left, right, bottom, top) in data coordinates
                    extent = [left, left+width, top+height, top]
                    
                    # Plot the image at specific coordinates
                    plt.imshow(image, extent=extent)

                    
                    
                    


        plt.xlim((0, df.resX.iloc[0]))
        plt.ylim((df.resY.iloc[0]), 0)

        plt.xlabel('Horizontal eye position (pixels)')
        plt.ylabel('Vertical eye position (pixels)')

        plt.legend((raw_h, fix_h), ('raw samples', 'fixations'), scatterpoints=1)

        # save figure
        plt.savefig(os.path.join(path, subj_nr+f'_2D{cond}.jpg'), dpi=1000, pad_inches=0)
        
        plt.close()

    
    

# Get latency of all fixations and their order in a trial

In [24]:
# Get fixation latency

def getFixationLatency(df):    

    # Get timestamp of when target was presented and add it to the dataframe

    # 1) get the first time sample when the target is presented
    sampTime_df = df.drop_duplicates(subset=['trialNr'],  keep='first', ignore_index=True)

    # 2)extract the columns needed
    sampTime_df = sampTime_df[['trialNr', 'sampTime']]

    # 3) rename the columns so they would be added
    sampTime_df.columns = ['trialNr', 'targSampTime']

    # 4) merge the target time into the main df (one time per trial)
    df = pd.merge(df, sampTime_df, on="trialNr")

    # Extract saccade latencies

    # 1) select only rows where fixation started
    fl_df = df[df.FixStartEnd == 'fix_start']

    # 2) select only rows with large enough preceeding saccade
#     fl_df = fl_df[fl_df.DistFromPrevFix > 300]

    # 3) compute first fixation duration (saccade latency)
    fl_df['FixLatency'] = fl_df.sampTime - fl_df.targSampTime

    # 4) remove rows where negative Saccade Latencies for trials where no fixation end is present
#     fl_df = fl_df[fl_df.FixLatency > 0]

    # Clip the negative values to zero. This ensures that fixations that carry over and do not have fix_start have a zero latency
    fl_df['FixLatency'] = fl_df['FixLatency'].clip(lower=0)

    
    # 5)
    # Initialize an empty list to hold the groups
    fixorder_groups = []
    
    for name, group in fl_df.groupby('trialNr'):
        
        # Add a new column with the order (rank) of the values
        # 'method='first'' ensures that the order respects the original order in case of ties
        group['FixationOrder'] = group['FixLatency'].rank()
        
        # Append the modified group to the list
        fixorder_groups.append(group)
        
#         print(f'Group: {name}')
#         print(group)
#         print()        
    
    # Concatenate all the modified groups back into a single DataFrame
    fl_df_modified = pd.concat(fixorder_groups)
    
    # Extract the columns needed
    fl_df_modified = fl_df_modified[['sampTime', 'FixLatency', 'FixationOrder']]
    
    # Filter out all rows except fix_start and fix_end
    df_start_end = df[df['FixStartEnd'].isin(['fix_start', 'fix_end'])]

    # Merge the variable into main df
    df_modified = pd.merge(df_start_end, fl_df_modified, on=["sampTime"], how="left")
    
    return df_modified  
    
# df_modified = getFixationLatency(df1)




In [25]:
def handle_carryover_fixations_and_merge(df):

    trial_duration = 10000 # maximum trial duration
    fixcarryover_groups = []
    
    #***STEP 1: Identify, label and correct carryover fixations ***
    # Loop thru trials and identify fixations that carry over the trials
    # There are two options:
    # 1) Fixation starts during previous trial or event and ends at the beginning of the trial (fix_end event without fix_start event)
    # 2) Fixation starts at the end of current trial and finishes after the trial is done (fix_start event without fix_end event)
    # If not corrected, this results in incorrect fixation durations, fixation latency, missing fixations 
    for name, group in df.groupby('trialNr'):
        fix_start = group[group.FixStartEnd == 'fix_start'].FixStartEnd
        fix_end = group[group.FixStartEnd == 'fix_end'].FixStartEnd

        print(fix_start.count(), fix_end.count())

        # if trial starts with fixation end, we need to add a fixation start event
        if group.FixStartEnd.iloc[0] == 'fix_end':
            print(f'Trial {name} starts with fix_end')

            # Add extra fixation start event

            # Reset variable in the first row
            group.FixStartEnd.iloc[0] = 'fix_end_carryover_inserted_start' # label such fixation event
            group.FixDur.iloc[0] = group.sampTime.iloc[0] - group.targSampTime.iloc[0] # recalculate fixation duration

            # Insert a fix_start event
            # Make a copy of the first row with a new index
            first_row = group.iloc[0:1].copy()
            first_row.index = [-1]  # Assign a negative index
            first_row.FixStartEnd = 'fix_start_carryover_inserted_start'
            first_row.FixDur = 0
            first_row.FixLatency = 0


            # Prepend the copied first row to the original DataFrame
            group = pd.concat([first_row, group])

            # Reset the index if you want a continuous numeric index
            group = group.sort_index().reset_index(drop=True)

            # Now we need to re-rank the order of fixations in the trial, since we added a new one in the beginning
            group['FixationOrder'] = group['FixLatency'].rank()


        # if trial ends with fixation start, we need to add a fixation end event
        if group.FixStartEnd.iloc[-1] == 'fix_start':
            print(f'Trial {name} ends with fix_start')

            # Add fixation end event
            # Reset variable in the last row
            group.FixStartEnd.iloc[-1] = 'fix_start_carryover_inserted_end'
            group.FixDur.iloc[-1] = 0

            # Insert a fix_start event
            # Make a copy of the last row with a ne index
            last_row = group.iloc[[-1]].copy()        
            last_row.index = last_row.index+1 # Assign the next index
            last_row.FixStartEnd = 'fix_end_carryover_inserted_end'
            last_row.FixDur = (last_row.targSampTime + trial_duration) - last_row.sampTime
            last_row.FixLatency = 0
            last_row.FixationOrder= 0

            # Append the copied first row to the original DataFrame
            group = pd.concat([group, last_row])

            # Reset the index if you want a continuous numeric index
            group = group.sort_index().reset_index(drop=True)                      



        # Accumulate groups into a list
        fixcarryover_groups.append(group)

    # Concatenate all the modified groups back into a single DataFrame
    fc_df = pd.concat(fixcarryover_groups)

    #*** STEP 2: Collapse all fixation events, such that all information is provided per each fixation****
    
    # *** 1. Merge all fixation events for fixations that happen within trial ***
    
    # Get only fix_start events
    df_fix_start = fc_df[fc_df.FixStartEnd == 'fix_start']
    # Drop the FixDur column, which should be empty for fix_start events
    df_fix_start = df_fix_start.drop('FixDur', axis=1)

    # Get only fix_end events
    df_fix_end = fc_df[fc_df.FixStartEnd == 'fix_end']
    # Select only the relevant events from fix_end events
    df_fix_end = df_fix_end[["FixXPos", "FixYPos", "FixDur"]]


    # Merge fix start and end for the same fixations
    df_merged = pd.merge(df_fix_start, df_fix_end, on=["FixXPos", "FixYPos"])

    #*** 2. Merge carryover fixation which missed the fix_start event ***
    
    # Get only fix_start_carryover_inserted_start events
    df_fix_start_insert_start = fc_df[fc_df.FixStartEnd == 'fix_start_carryover_inserted_start']
    df_fix_start_insert_start = df_fix_start_insert_start.drop('FixDur', axis=1)

    # Get only fix_end_carryover_inserted_start events
    df_fix_end_insert_start = fc_df[fc_df.FixStartEnd == 'fix_end_carryover_inserted_start']
    # Select only the relevant events from fix_end events
    df_fix_end_insert_start = df_fix_end_insert_start[["FixXPos", "FixYPos", "FixDur"]]

    df_merged_insert_start = pd.merge(df_fix_start_insert_start, df_fix_end_insert_start, on=["FixXPos", "FixYPos"])

    #*** 3. Merge carryover fixations which missed the fix_end event ***
    
    # Get only fix_start_carryover_inserted_end events
    df_fix_start_insert_end = fc_df[fc_df.FixStartEnd == 'fix_start_carryover_inserted_end']
    df_fix_start_insert_end = df_fix_start_insert_end.drop('FixDur', axis=1)

    # Get only fix_end_carryover_inserted_end events
    df_fix_end_insert_end = fc_df[fc_df.FixStartEnd == 'fix_end_carryover_inserted_end']
    # Select only the relevant events from fix_end events
    df_fix_end_insert_end = df_fix_end_insert_end[["FixXPos", "FixYPos", "FixDur"]]

    df_merged_insert_end = pd.merge(df_fix_start_insert_end, df_fix_end_insert_end, on=["FixXPos", "FixYPos"])

    #*** 4. Concatenate all carryover fixations***
    # Now concatenate all carryover fixations, inserted_start and inserted_end
    df_carryover = pd.concat([df_merged_insert_start, df_merged_insert_end], ignore_index=True)

    # Sort the combined DataFrame based on frameNr
    df_carryover = df_carryover.sort_values(by='frameNr')

    
    # *** 5. Now concatenate carryover fixations with within_trial fixations ***
    df_final = pd.concat([df_merged, df_carryover], ignore_index=True)
    # Sort based on frameNr
    df_final = df_final.sort_values(by='frameNr')


    return df_final

# df_final = handle_carryover_fixations_and_merge(df_modified)    
    

# Calculating AOI for each fixation

In [26]:
def addAOI(df):  
    
    """
    df should contain only fixation events, no raw data
    """
    
    def is_point_in_box(point, box):
        """
        Determine if a point is within a bounding box.

        Parameters:
        - point: A tuple (x, y) representing the point.
        - box: A tuple ((x1, y1), (x2, y2)) representing the bounding box, 
               where (x1, y1) is the top-left corner and (x2, y2) is the bottom-right corner.

        Returns:
        - True if the point is within the box, False otherwise.
        """
        px, py = point
        (x1, y1), (x2, y2) = box

        return x1 <= px <= x2 and y1 <= py <= y2

    def get_bounding_box_assignment(boxes, point):
        """
        Determine the bounding box a point belongs to.

        Parameters:
        - boxes: A list of tuples representing the bounding boxes.
                 Each bounding box is defined as ((x1, y1), (x2, y2)).
        - point: A tuple (x, y) representing the point.

        Returns:
        - The index of the bounding box the point belongs to, or None if it doesn't belong to any boxes.
        """
        for i, box in enumerate(boxes):
            if is_point_in_box(point, box):
                return i
        
        return 'None'


    import ast
    width = 250
    height = 250
    imageName = 'None'
    bbox_assignments = []
    stim_assignments = []

    aoi_df = df

    for index, row in aoi_df.iterrows():
        
        if row.stimuli == 'dresses':
            height = 375            
        
        # Get all stimuli names for this trial
        imageNames = row.imageNames
        
        # Convert str to actual list
        imageNames = ast.literal_eval(imageNames)

        # Bounding boxes for this trial
        bounding_boxes = []

        # Get coordinates of stimuli for this fixation
        left_top_coords = row.imageLocations

        # Convert string to actual list
        left_top_coords = ast.literal_eval(left_top_coords)

        # Iterate over bboxes
        for coord in left_top_coords:      
            # Assemble coordinates for the bounding boxes
            x1 = coord[0]
            y1 = coord[1]
            x2 = coord[0] + width
            y2 = coord[1] + height
            bounding_boxes.append([(x1,y1), (x2,y2)])        

        # Get fixation coordinates
        point = (row.FixXPos, row.FixYPos)
        # get the index of the AOI where this fixation point falls
        assignment = get_bounding_box_assignment(bounding_boxes, point)

        # if fixation in the bounding box, get the stimulus name for this bounding box
        if assignment != 'None':
            imageName = imageNames[assignment].split('/')[-1]
        else:
            # To specify None, add the stimuli event 
            imageName = 'None_' + row.stimuli
            assignment = assignment + '_' + row.stimuli


#         print(f'The point {point} belongs to bounding box: {assignment}, image:{imageName}')


        # Accumulate assignements
        bbox_assignments.append(assignment)
        stim_assignments.append(imageName)

    aoi_df['AOI_bbox'] = bbox_assignments
    aoi_df['AOI_stim'] = stim_assignments
    
    # Reset index
    aoi_df = aoi_df.reset_index(drop=True)
       
    
    return aoi_df

# a = addAOI(df_final)
# a.to_csv(path +'/sample_participant_VBLExp1.csv', index=False)

# Analyze all subjects

In [27]:
# Path to data folders

# path = 'C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved'
path = 'D:/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved'

def create_directory_if_not_exists(directory_path):
    try:
        os.makedirs(directory_path)
        print(f"Directory '{directory_path}' was created.")
    except FileExistsError:
        # The directory already exists, no need to create it.
        print(f"Directory '{directory_path}' already exists.")

# definde data ana analysis directories and create them if they don't exist yet
path_to_data = os.path.join(path, 'data')
path_to_analysis = os.path.join(path, 'analysis')
create_directory_if_not_exists(path_to_analysis)

output_dfs = []
# get all folder names
folder_names = os.listdir(path_to_data)

# read and process datafile with fixations (_extra) for each participant
for fn in folder_names:
    path_to_file = os.path.join(path_to_data, fn, fn+'_record_extra.csv')
    
    print(f'Processing participant {fn}...')
        
    try:
        df = pd.read_csv(path_to_file)        
    except:
        print('File does not exist: ' + path_to_file)
        continue
        
    # Extract only samples when the target was presented
    df1 = df[df.event=='target_on']
    
    try:
        plot2d(df1, fn, path_to_analysis, bboxes=True, stimuli=True)
    except ValueError:
        continue
        
    df1 = getFixationLatency(df1)
    df1 = handle_carryover_fixations_and_merge(df1)
    df1 = addAOI(df1)
    
    # Accumulate analyzed data across participants
    output_dfs.append(df1)
    
output_df = pd.concat(output_dfs)
output_df.to_csv(os.path.join(path_to_analysis, 'allSubjects_VBLExp1.csv'), index=False)



Directory 'D:/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved\analysis_tmp' was created.
Processing participant 2024_02_19_08_03_20...


C:\Users\Artem\AppData\Local\Temp\ipykernel_1052\1438454985.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df.sampTime - fl_df.targSampTime
C:\Users\Artem\AppData\Local\Temp\ipykernel_1052\1438454985.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df['FixLatency'].clip(lower=0)
C:\Users\Artem\AppData\Local\Temp\ipykernel_1052\2621730654.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

18 19
Trial 0 starts with fix_end
15 15
12 13
Trial 2 starts with fix_end
14 15
Trial 3 starts with fix_end
16 17
Trial 4 starts with fix_end
Processing participant 2024_02_19_08_03_44...


C:\Users\Artem\AppData\Local\Temp\ipykernel_1052\1438454985.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df.sampTime - fl_df.targSampTime
C:\Users\Artem\AppData\Local\Temp\ipykernel_1052\1438454985.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df['FixLatency'].clip(lower=0)
C:\Users\Artem\AppData\Local\Temp\ipykernel_1052\2621730654.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

16 17
Trial 0 starts with fix_end
15 15
Trial 1 starts with fix_end
Trial 1 ends with fix_start
19 19
13 14
Trial 3 starts with fix_end
18 18
Processing participant 2024_02_19_08_04_13...
21 22
Trial 0 starts with fix_end
20 21
Trial 1 starts with fix_end
16 17
Trial 2 starts with fix_end
15 16
Trial 3 starts with fix_end
15 15


C:\Users\Artem\AppData\Local\Temp\ipykernel_1052\1438454985.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df.sampTime - fl_df.targSampTime
C:\Users\Artem\AppData\Local\Temp\ipykernel_1052\1438454985.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df['FixLatency'].clip(lower=0)
C:\Users\Artem\AppData\Local\Temp\ipykernel_1052\2621730654.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

# Summary

In [28]:
path = 'C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/Young/Approved'
path_to_analysis = os.path.join(path, 'analysis')

output_df = pd.read_csv(os.path.join(path_to_analysis, 'allSubjects_NoveltyIndex.csv'))

# filter out excluded participants
# '2024_01_15_14_19_20' was the second time, '2024_01_26_17_16_28' too few frames
output_df = output_df[~output_df['deepeye-id'].isin(['2024_01_15_14_19_20', '2024_01_26_17_16_28'])]

# group data
total_count = output_df.groupby(['pp_id', 'deepeye-id']).noveltyIdx_fixCountProp.count()
fixCountProp = output_df.groupby(['pp_id', 'deepeye-id']).noveltyIdx_fixCountProp.mean()
fixDurProp = output_df.groupby(['pp_id', 'deepeye-id']).noveltyIdx_fixDurProp.mean()


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/Young/Approved\\analysis\\allSubjects_NoveltyIndex.csv'